In [ ]:
# 1 Import libraries
 
import os 
import numpy as np
import skimage as ski
import skimage.feature
import matplotlib.pyplot as plt
from PIL import Image
import random
from scipy.io import loadmat
from scipy.io import savemat
import math
import copy
import av
import os 
import numpy as np
import skimage as ski
import skimage.feature
import matplotlib.pyplot as plt
from PIL import Image
import random

scale_source = 0.2 + np.round(np.logspace(np.log10(0.20), np.log10(0.8), num=15), 3)
path = 'E:/Google Drive/Research/Dry Mass Research/TBI Stitched Data/'
name = ['5', '10', '15', '20']
timeh = name[0]
number_of_images = len(name)


In [ ]:
#3 Define functions
from collections import Counter

def filter_regions(func, image, use_coords=False, background_label=0):
    """Removes regions from the image for which func returns True"""

    # Filter regions by properties
    for region in ski.measure.regionprops(image):
        if func(region) == False:
            continue

        if use_coords:
            # Delete pixel by pixel
            for row, col in region.coords:
                image[row, col] = background_label
        else:
            # Delete using indexing
            image[image == region.label] = background_label

    return image


def relabel(image, background_label=0):
    image[image > 0] = 1
    image = ski.measure.label(image, background=background_label)

    return image

def update_labels(image, ref, background_label=0):

    neurons = {}

    # Check if the neurons did not merge
    for region in ski.measure.regionprops(ref):

        # Get the pixel labels in the image
        labels = np.take(image, np.ravel_multi_index(np.array(region.coords).T, image.shape))
        neurons[region.label] = Counter(labels[labels.nonzero()])

    return neurons

def extract_traces(image, eigen_threshold=0.3, hessian_sigma=4, image_threshold=0.2, median_disk_size=3):
#def extract_traces(image, eigen_threshold=0.15, hessian_sigma=3.5, image_threshold=0.55, median_disk_size=2):

    # Preprocess the image: sharpen and threshold
    image = ski.filters.median(image, ski.morphology.disk(median_disk_size)) #
    image = ski.img_as_float(image) < image_threshold

    # Calculate the eigenvalues of the Hessian matrix
    Hrr, Hrc, Hcc = ski.feature.hessian_matrix(image, sigma=hessian_sigma, order='rc')

    D = np.sqrt(16/9 * (Hrr - Hcc)**2 + 16/9 * 4 * Hrc**2)
    E1 = 2/3 * (Hrr + Hcc) + D
    E2 = 2/3 * (Hrr + Hcc) - D

    return E1 / abs(E2.min()) > eigen_threshold

In [ ]:
# for i in range(0, number_of_images):
    
#     image_mat = np.asarray(Image.open(path + 'f0_t' + name[i] + '_mSLIM_stitched.tif'))
   
#     savemat(path + 'f0_t' + name[i] + '_mSLIM_stitched.mat', {'image':image_mat}) 
    
#     mass_mat = image_mat
#     constant=(0.58)/(2*math.pi*(0.2))*(0.27)**2
   
#     mass_mat = mass_mat*constant
#     mass_mat[np.where(mass_mat < 0)] = 0 
    
#     np.save(path + 'f0_t' + name[i] + '_mSLIM_stitched_mass.npy', mass_mat)

In [ ]:


for i in range(0, len(scale_source)):
    
    scale = [0.2, scale_source[i]]
    print(scale)
    

    
    jpg_name =  'f0_t' + timeh + '_mSLIM_stitched.jpg'
    
#     t = 't14' #XC: times series
    saved_name = 'f0_t' + timeh + '_mSLIM_stitched' #XC: saved name
#     saved_name_jpg = t+'_c_012345_r_234567' #XC :{ , some small issues for the jpgs
#     name = t+'_c_012345_r_2234567'+'_image' #image name
#     jpg_name = saved_name_jpg+'_image'
    scale_str = '('+str(scale[0])+'_'+str(scale[1])+','+str(scale[0])+'_'+str(scale[1])+')'


    #2: open the RGB image
    #XC: open a specific image

    frame_list = []
    #A1 = Image.open('bottom-right-t1.jpg')
    A1 = Image.open(path + 'JPG/' + jpg_name)
    
    
    a1=np.asarray(A1)
    a2=a1[int(round(a1.shape[0]*scale[0])):int(round(a1.shape[0]*scale[1])), int(round(a1.shape[1]*scale[0])):int(round(a1.shape[1]*scale[1]))]
    B1=ski.color.rgb2gray(a2)

    frame_list.append(B1)
    #XC: show the image in graysacle

    #plt.figure(figsize=[25, 25]) #original
    #plt.figure(figsize=[10, 10])
    #plt.imshow(B1)
    #plt.colorbar()


    

    #4 Get labeled neurons
    from skimage.segmentation import random_walker

    labeled_list = []

    for i, frame in enumerate(frame_list[0:1]): #XC: enumerate every pictures 

        markers = np.zeros(shape=frame.shape, dtype=np.uint) #XC: initialise the markers
        #markers[frame < 0.52] = -1 #XC: background--underexposure
        #markers[frame > 0.6] = 1 #XC: background--overexposure
        markers[frame < 0.1] = -1 #XC: background--underexposure
        markers[frame > 0.58] = 1 #XC: background--overexposure


        # Run random walker algorithm
        I_neurons = random_walker(frame, markers, beta=1, mode='cg_mg') 

        #XC: use the random walker algorithm to segment the picture
        #XC: https://blog.csdn.net/menjiawan/article/details/46862115

        # Label and remove small patches
        I = ski.measure.label(I_neurons, background=2) #XC: 'backgroud=2' !!?!!

        #XC: use ski.measure.label to label all connected regions 
        #XC: https://www.jianshu.com/p/1b90b549b50e  
        #XC: https://www.cnblogs.com/denny402/p/5166258.html

        I = filter_regions(lambda x: x.area < 185, I) #XC: delete regions that are too small （less than 800 pixels）
        #plt.figure(figsize=[12, 12])
        #plt.imshow(I)
        #plt.colorbar()
        if i >= 0: #XC: main
            # Relabel to have consecutive IDs
            ref = relabel(I) #XC: reconstruct the matrix 
        else: #XC: why i can be less than 0
            neurons = update_labels(I, ref)
            ref = np.zeros_like(I) #XC: generate a zero matrix that has the shape of I

            for ref_label, counts in neurons.items():

                if len(counts) >= 1:
                    label_to_replace = counts.most_common(1)[0][0]
                    ref[I == label_to_replace] = ref_label

        labeled_list.append(ref)
        print("Processed {:d} out of {:d} images".format(i + 1, len(frame_list)))

        plt.figure(figsize=[12, 12])
        plt.imshow(labeled_list[0])
        plt.colorbar()
        plt.show()

    #5 Keep only the traces that touch neurons            
    res_list=[]
    import matplotlib.patches as mpatches
    num_of_label=0
    for i, frame in enumerate(frame_list[0:1]):
        I = extract_traces(frame, 0.0525, 6.0, 0.08, 6.0) #XC: find traces
        I = I.astype(np.uint8) 
        I = ski.measure.label(I, background=0)

        res = np.zeros_like(I)

        I_neurons = labeled_list[i]
        I_neurons = ski.morphology.dilation(I_neurons, ski.morphology.disk(21))
        #XC: https://blog.csdn.net/limiyudianzi/article/details/86981126

        # Check if the neurons did not merge
        for region in ski.measure.regionprops(I):

            # Get the pixel labels in the image
            if np.take(I_neurons, np.ravel_multi_index(np.array(region.coords).T, I.shape)).any():
                res[I == region.label] = -1
        #I_mid=np.zeros_like(res)
        #I_mid[res*labeled_list[0]!=0]=1
        #new_image=res+labeled_list[0]-I_mid
        res_list.append(res)
        plt.figure(figsize=[25, 25])
        plt.imshow(ski.color.label2rgb(res_list[0], image=frame_list[0], bg_label=0, alpha=0.2, colors=[(0.0, 1.0, 0.0)]))
        plt.imshow(labeled_list[0] + 100, alpha=0.35)
        plt.show()
    #XC: convert label to rgb



    # XC: Weighted Version (2/3)

    #6 Construct blocks and label edges
    import copy
    from time import time
    #Construct blocks####
    ######################
    neuron_location=[]


    ##########
    adjacent_m_list = []
    new_image_list = []
    num_of_label_list = []
    MAX_GAPS_NUM = 2
    TEST_NEU = 433  #XC: good example: 433, 633, 259
    TEST_TRA = 3 #XC: good example 433/3
    TEST_SWITCH_R = False 
    TEST_SWITCH_T = False
    LENGTH = 200 + a2.shape[1]/20
    #***************************************big loop
    for j, labeled in enumerate(labeled_list): #XC: enumerate every image 

        num_of_label = 0
        hm = 0
        wm = 0
        for region in ski.measure.regionprops(labeled):
            hm += 1 #XC: number of neurons
            wm += 1 #XC: number of neurons

        height = labeled.shape[0] #XC: heigh of the image 
        width = labeled.shape[1] #XC: width of the image 

        #XC: initialise adjacent_m
        adjacent_m = np.zeros((hm + 1, wm + 1)) - 1000
        adjacent_num= np.zeros((hm + 1, wm + 1)) 
        print('Processing Image', j) #XC: j is the image number
        print('Number of Neurons', hm) #XC: hm is the number of neurons

        for i in range(0, hm + 1):
            adjacent_m[i][0] = i

        for i in range(0, wm + 1):
            adjacent_m[0][i] = i

        minr_list = [] #XC: min row
        minc_list = [] #XC: min column
        maxr_list = [] #XC: max row
        maxc_list = [] #XC: max column
        max_label = 0 #XC: the max label number

        new_image = res_list[j] + labeled #XC: new image = traces + neurons

        for region in ski.measure.regionprops(labeled):
            num_of_label += 1 #XC: number of label
            max_label = region.label #XC: the max label number
        rect_label = []
        rect_label_3D = []
        '''
        for i in range(0,labeled.shape[0]):
            for j in range(0,labeled.shape[1]):
                if new_image[i][j]>max_label:
                    new_image[i][j]=max_label
        '''


        for region in ski.measure.regionprops(labeled): #XC: enumerate every region

            if region.area >= 185: #XC: if the area of the neurons > 800 pixels
                minr,minc,maxr,maxc = region.bbox #XC: region.bbox = (min_row, min_col, max_row, max_col)

                #XC: construct the database of the range of ragions
                minr_list.append(minr) 
                minc_list.append(minc)
                maxr_list.append(maxr)
                maxc_list.append(maxc)

            #**************************************************** label all the red frames
                rect_la = []
            #*****************************************************label blocks. 
                #XC: expands by 15 pixels in all directions
                mina = minc - 5
                maxa = maxc + 5
                minb = minr - 5
                maxb = maxr + 5
            #*******************************************determine blocksize. 

                #XC: prevent out of scope
                if maxa >= labeled.shape[1]:
                    maxa = labeled.shape[1] - 1 
                if maxb >= labeled.shape[0]:
                    maxb = labeled.shape[0] - 1
                if mina < 0:
                    mina = 0
                if minb < 0:
                    minb = 0
                ###############
                neuronbuf=[(maxa-mina)/2, (maxb-minb)/2]
                neuron_location.append(neuronbuf)
                #################

                #XC: a = column, b = row

                #XC: -----------------------------------------------------------
                #XC: enumerate every pixel of the blocks


                #********************************************
                #XC: bottom 
                for i in range(mina, maxa + 1):

                    if new_image[minb][i] == -1: #XC: if there is a trace
                        new_image[minb][i] = region.label + num_of_label #XC: relabel the region
                        buf = [minb, i] #XC: record position
                        rect_la.append(buf)

                    else:
                        #XC: cope with overlapping areas 
                        if new_image[minb][i] > num_of_label and new_image[minb][i] != region.label + num_of_label:
                           #XC: 1. if the pixel has been labeled; 2. if this pixel also belongs to another region

                            #XC: the two neurons are connected with each other
                            adjacent_m[region.label][new_image[minb][i] - num_of_label] = -1 
                            adjacent_m[new_image[minb][i] - num_of_label][region.label] = -1
                            #######
                            adjacent_num[region.label][new_image[minb][i] - num_of_label]= adjacent_num[region.label][new_image[minb][i] - num_of_label]+1
                            adjacent_num[new_image[minb][i] - num_of_label][region.label]=adjacent_num[new_image[minb][i] - num_of_label][region.label]+1
                            ######

                        else:

                            #XC: the pixel belongs to this region
                            new_image[minb][i] = region.label + num_of_label

                #********************************************
                #XC: top  

                for i in range(mina, maxa + 1):

                    if new_image[maxb][i] == -1:
                        new_image[maxb][i] = region.label + num_of_label
                        buf = [maxb, i]
                        rect_la.append(buf)

                    else:

                        if new_image[maxb][i] > num_of_label and new_image[maxb][i] != region.label + num_of_label:
                            adjacent_m[region.label][new_image[maxb][i] - num_of_label] = -1
                            adjacent_m[new_image[maxb][i] - num_of_label][region.label] = -1
                            #######
                            adjacent_num[region.label][new_image[maxb][i] - num_of_label]= adjacent_num[region.label][new_image[maxb][i] - num_of_label]+1
                            adjacent_num[new_image[maxb][i] - num_of_label][region.label]= adjacent_num[new_image[maxb][i] - num_of_label][region.label]+1
                            ########
                        else:
                            new_image[maxb][i] = region.label + num_of_label

                #********************************************
                #XC: left 

                for i in range(minb, maxb + 1):

                    if new_image[i][mina] == -1:
                        new_image[i][mina] = region.label + num_of_label
                        buf = [i, mina]
                        rect_la.append(buf)

                    else:

                        if new_image[i][mina] > num_of_label and new_image[i][mina] != region.label + num_of_label:
                            adjacent_m[region.label][new_image[i][mina] - num_of_label] = -1
                            adjacent_m[new_image[i][mina] - num_of_label][region.label] = -1
                            #######
                            adjacent_num[region.label][new_image[i][mina] - num_of_label] = adjacent_num[region.label][new_image[i][mina] - num_of_label]+1
                            adjacent_num[new_image[i][mina] - num_of_label][region.label] = adjacent_num[new_image[i][mina] - num_of_label][region.label]+1
                            ########

                        else:
                            new_image[i][mina] = region.label + num_of_label
                #********************************************
                #XC: right 
                for i in range(minb, maxb + 1):

                    if new_image[i][maxa] == -1:
                        new_image[i][maxa] = region.label + num_of_label
                        buf = [i, maxa]
                        rect_la.append(buf)

                    else:

                        if new_image[i][maxa] > num_of_label and new_image[i][maxa] != region.label + num_of_label:
                            adjacent_m[region.label][new_image[i][maxa] - num_of_label] = -1
                            adjacent_m[new_image[i][maxa] - num_of_label][region.label] = -1   
                            ########
                            adjacent_num[region.label][new_image[i][maxa] - num_of_label] = adjacent_num[region.label][new_image[i][maxa] - num_of_label]+1
                            adjacent_num[new_image[i][maxa] - num_of_label][region.label] = adjacent_num[new_image[i][maxa] - num_of_label][region.label]+1                                                                                                                 
                            #####

                        else:
                            new_image[i][maxa] = region.label + num_of_label
                #**************************************************************
                rect_label.append(rect_la)


                #XC: reconstruct the database of startpoints.

                group = []
                group_marker = [0, 0]
                rect_la_3D = []
                for i, startpoint in enumerate(rect_la):

                    if np.abs(startpoint[0] - group_marker[0]) + np.abs(startpoint[1] - group_marker[1]) < 2:
                        group.append(startpoint)
                        group_marker = startpoint

                    else:
                        group_marker = startpoint
                        if len(group) > 0:
                            rect_la_3D.append(group)
                        group = []
                        group.append(startpoint)


                if len(group) > 0:
                    rect_la_3D.append(group)
                rect_label_3D.append(rect_la_3D)

                #***************************************************
        ###################






        print('Blocks Created\n')






        #XC: detect edges
        print('Detecting edges')
        start = time()
        #print(adjacent_m)
        #plt.figure(figsize=[25, 25])
        #plt.imshow(new_image)
        #plt.colorbar()
        ###################################################print edges

        #XC: what is deep copy: https://blog.csdn.net/u010712012/article/details/79754132

        tem_label = copy.deepcopy(rect_label_3D)  #XC: rect_label = startpoints (2D list)
        new_ima = copy.deepcopy(new_image) #XC: new_ima/new_image = labeled blocks + traces (-1)
        endp = [] #endpoints
        counter_num = np.zeros(np.shape(new_ima)) - 1000 #XC: visualise counter map


        #XC: I. enumerate every region
        for region in ski.measure.regionprops(labeled):

            #XC: detect the connection of a specific neurons
            if TEST_SWITCH_R and region.label != TEST_NEU:
                continue
            print('\rProcessing %d out of %d' %(region.label, hm), end='')
            b = region.label - 1 
            tem_lab = copy.deepcopy(tem_label[b]) #XC: startpoints in current region
            new_ima = copy.deepcopy(new_image) #XC: a copy of the whole graph

            for k, traces in enumerate(tem_lab):
                if TEST_SWITCH_T and k != TEST_TRA:
                    continue
                counter = 0
                bt_lab = copy.deepcopy(traces)  #XC: bt_lab is tem_lab  
                gap_num = np.zeros(np.shape(new_ima)) #XC: a zero matrix that have the shape of the graph
                for point in traces:
                    gap_num[point[0], point[1]] = 1 #XC: mark all start point with 1
                i = 0
                flag_hasNew = True
                #XC: II. the number of gap times 
                for nowGapsNum in range(1, MAX_GAPS_NUM):
                    nexpGap_lab = []
                    flag_hasNew = True
                    #:XC: III. find endpoints of traces
                    while flag_hasNew and i < LENGTH:   #iteration
                        counter += 1;
                        #XC: this length also set a limit of gap times!
                        flag_hasNew = False
                        i += 1
                        j = 0
                        #XC: IV: enumerate every start point
                        for m in traces:
                            x = m[0]
                            y = m[1]
                            #print("j=",j)
                            #print("x=",x,"y=",y)
                            bt_lab.remove(m) #XC: https://www.runoob.com/python/att-list-remove.html
                            #XC: remove the start point that has been detected
                            j += 1 #XC: j = number of startpoints that have been proceeded
                            #*******************************************find endpoints  
                            #XC: 
                            #XC: elif: cope with overlapping regions 
                            #XC: set a safe frame
                            if 10 < x < labeled.shape[0] - 10 and 10 < y < labeled.shape[1] - 10:
                                cnt = 0
                                shouldo = True
                                bound = [new_ima[x+3][y],new_ima[x-3][y],new_ima[x][y+3],new_ima[x][y-3]]
                                #XC: 4 pixels at the left/right/bottom/top
                                for elem in bound: 
                                    if elem != 0: #XC: if the pixel is not background
                                        cnt += 1 
                                    if cnt > 1: #XC: if there are at least two pixel =! 0 
                                        shouldo = False
                                        break
                                #XC: if the pixel is at the corner of a region
                                if shouldo:
                                    endp.append((x,y)) 
                                    #XC: a 10*10 block
                                    for r in range(x - 4, x + 6, 2):
                                        if new_ima[r][y - 5] == -1:
                                            new_ima[r][y - 5] = region.label + num_of_label
                                            gap_num[r, y - 5] = nowGapsNum + 1
                                            counter_num[r, y - 5] = counter + 5
                                            nexpGap_lab.append((r, y - 5))
                                            #flag_hasNew = True
                                        elif new_ima[r][y - 5] > num_of_label and new_ima[r][y - 5] != region.label + num_of_label:
                                            #XC: adjacent regions
                                            if adjacent_m[region.label][new_ima[r][y - 5] - num_of_label] == -1000 or adjacent_m[region.label][new_ima[r][y - 5] - num_of_label] > counter + 5:
                                                adjacent_m[region.label][new_ima[r][y - 5] - num_of_label] = counter + 5
                                                adjacent_m[new_ima[r][y - 5]-num_of_label][region.label] = counter + 5
                                                ############
                                                ##adjacent_num[region.label][new_ima[r][y - 5] - num_of_label] = adjacent_num[region.label][new_ima[r][y - 5] - num_of_label] + 1
                                                ##adjacent_num[new_ima[r][y - 5]-num_of_label][region.label] = adjacent_num[new_ima[r][y - 5]-num_of_label][region.label] + 1
                                                #######

                                    for r in range(x - 6, x + 4, 2):
                                        if new_ima[r][y + 5] == -1:
                                            new_ima[r][y + 5] = region.label + num_of_label
                                            gap_num[r, y + 5] = nowGapsNum + 1
                                            counter_num[r, y + 5] = counter + 5
                                            nexpGap_lab.append((r, y + 5))
                                            #flag_hasNew = True
                                        elif new_ima[r][y + 5] > num_of_label and new_ima[r][y + 5] != region.label + num_of_label:
                                            if adjacent_m[region.label][new_ima[r][y + 5] - num_of_label] == -1000 or adjacent_m[region.label][new_ima[r][y + 5] - num_of_label] > counter + 5:
                                                adjacent_m[region.label][new_ima[r][y + 5] - num_of_label] = counter + 5
                                                adjacent_m[new_ima[r][y + 5] - num_of_label][region.label] = counter + 5  
                                                ######
                                                ##adjacent_num[region.label][new_ima[r][y + 5] - num_of_label] = adjacent_num[region.label][new_ima[r][y + 5] - num_of_label] + 1
                                                ##adjacent_num[new_ima[r][y + 5] - num_of_label][region.label] = adjacent_num[new_ima[r][y + 5] - num_of_label][region.label] + 1                                                                                           
                                                ###########                            
                                    for c in range(y - 4, y + 6, 2):
                                        if new_ima[x - 5][c] == -1:
                                            new_ima[x - 5][c] = region.label + num_of_label
                                            gap_num[x - 5, c] = nowGapsNum + 1
                                            counter_num[x - 5, c] = counter + 5
                                            nexpGap_lab.append((x - 5, c))
                                            #flag_hasNew = True
                                        elif new_ima[x - 5][c] > num_of_label and new_ima[x - 5][c] != region.label + num_of_label:
                                            if adjacent_m[region.label][new_ima[x - 5][c] - num_of_label] == -1000 or adjacent_m[region.label][new_ima[x - 5][c] - num_of_label] > counter + 5:
                                                adjacent_m[region.label][new_ima[x - 5][c] - num_of_label] = counter + 5
                                                adjacent_m[new_ima[x - 5][c] - num_of_label][region.label] = counter + 5
                                            ######
                                                ##adjacent_num[region.label][new_ima[x - 5][c] - num_of_label] = adjacent_num[region.label][new_ima[x - 5][c] - num_of_label] + 1
                                                ##adjacent_num[new_ima[x - 5][c] - num_of_label][region.label] = adjacent_num[new_ima[x - 5][c] - num_of_label][region.label] + 1                                                                                           
                                            ########

                                    for c in range(y - 6, y + 4, 2):
                                        if new_ima[x + 5][c] == -1:
                                            new_ima[x + 5][c] = region.label + num_of_label
                                            gap_num[x + 5, c] = nowGapsNum + 1
                                            counter_num[x + 5, c] == counter + 5
                                            nexpGap_lab.append((x + 5, c))
                                            #flag_hasNew = True
                                        elif new_ima[x + 5][c] > num_of_label and new_ima[x + 5][c] != region.label + num_of_label:
                                            if adjacent_m[region.label][new_ima[x + 5][c] - num_of_label] == -1000 or adjacent_m[region.label][new_ima[x + 5][c] - num_of_label] > counter + 5:
                                                adjacent_m[region.label][new_ima[x + 5][c] - num_of_label] = counter + 5
                                                adjacent_m[new_ima[x + 5][c] - num_of_label][region.label] = counter + 5
                                             ######
                                                ##adjacent_num[region.label][new_ima[x + 5][c] - num_of_label] = adjacent_num[region.label][new_ima[x + 5][c] - num_of_label] + 1
                                                ##adjacent_num[new_ima[x + 5][c] - num_of_label][region.label] = adjacent_num[new_ima[x + 5][c] - num_of_label][region.label] + 1                                                                                           
                                            ########                                                                            
                            #******************************************find endpoints
                            #XC: find endpoints by detecting the adjacent '-1's (Iterative Method)       
                            if x + 1 < labeled.shape[0]:
                                if new_ima[x + 1][y] == -1:
                                    new_ima[x + 1][y] = region.label + num_of_label
                                    gap_num[x + 1,y] = nowGapsNum
                                    counter_num[x + 1, y] = counter
                                    if (x+1,y) in nexpGap_lab:
                                        nexpGap_lab.remove((x + 1,y))
                                    bt_lab.append((x + 1,y)) #XC: append a new start point
                                    flag_hasNew = True
                                #print(len(bt_lab))
                                    #if m in bt_lab:
                                        #bt_lab.remove(m)
                                elif new_ima[x + 1][y] > num_of_label and new_ima[x + 1][y] != region.label + num_of_label:
                                    if adjacent_m[region.label][new_ima[x + 1][y] - num_of_label] == -1000 or adjacent_m[region.label][new_ima[x + 1][y] - num_of_label] > counter:
                                        adjacent_m[region.label][new_ima[x + 1][y] - num_of_label] = counter
                                        adjacent_m[new_ima[x + 1][y] - num_of_label][region.label] = counter
                                        #######
                                        adjacent_num[region.label][new_ima[x + 1][y] - num_of_label] =adjacent_num[region.label][new_ima[x + 1][y] - num_of_label]+1
                                        adjacent_num[new_ima[x + 1][y] - num_of_label][region.label] =adjacent_num[new_ima[x + 1][y] - num_of_label][region.label]+1                                                                                                   
                                        #######

                            if x - 1 >= 0:        
                                if new_ima[x - 1][y] == -1:
                                    new_ima[x - 1][y] = region.label + num_of_label

                                    gap_num[x - 1,y] = nowGapsNum
                                    counter_num[x - 1, y] = counter
                                    if (x - 1, y) in nexpGap_lab:
                                        nexpGap_lab.remove((x - 1, y))
                                    bt_lab.append((x - 1, y))
                                    flag_hasNew = True
                                    #if m in bt_lab:
                                        #bt_lab.remove(m)
                                #print(len(tem_lab))
                                elif new_ima[x - 1][y] > num_of_label and new_ima[x - 1][y] != region.label + num_of_label:
                                #print ("connect", region.label, new_ima[x-1][y]-15)
                                    if adjacent_m[region.label][new_ima[x - 1][y] - num_of_label] == -1000 or adjacent_m[region.label][new_ima[x - 1][y] - num_of_label] > counter:
                                        adjacent_m[region.label][new_ima[x - 1][y] - num_of_label] = counter
                                        adjacent_m[new_ima[x - 1][y] - num_of_label][region.label] = counter
                                        #######
                                        adjacent_num[region.label][new_ima[x - 1][y] - num_of_label] = adjacent_num[region.label][new_ima[x - 1][y] - num_of_label]+1
                                        adjacent_num[new_ima[x - 1][y] - num_of_label][region.label] = adjacent_num[new_ima[x - 1][y] - num_of_label][region.label]+1           
                                        #######


                            if y + 1 < labeled.shape[1]:        
                                if new_ima[x][y + 1] == -1:
                                    new_ima[x][y + 1] = region.label + num_of_label

                                    gap_num[x, y + 1] = nowGapsNum
                                    counter_num[x, y + 1] = counter
                                    if (x, y + 1) in nexpGap_lab:
                                        nexpGap_lab.remove((x, y + 1))
                                    bt_lab.append((x, y + 1))
                                    flag_hasNew = True
                                    #if m in bt_lab:
                                        #bt_lab.remove(m)
                                #print(len(tem_lab))
                                elif new_ima[x][y + 1] > num_of_label and new_ima[x][y + 1] != region.label + num_of_label:
                                    if adjacent_m[region.label][new_ima[x][y + 1] - num_of_label] == -1000 or adjacent_m[region.label][new_ima[x][y + 1] - num_of_label] > counter:
                                        adjacent_m[region.label][new_ima[x][y + 1] - num_of_label] = counter
                                        adjacent_m[new_ima[x][y + 1] - num_of_label][region.label] = counter
                                        #######
                                        adjacent_num[region.label][new_ima[x][y+1] - num_of_label] = adjacent_num[region.label][new_ima[x][y+1] - num_of_label]+1
                                        adjacent_num[new_ima[x][y+1] - num_of_label][region.label] = adjacent_num[new_ima[x][y+1] - num_of_label][region.label]+1           
                                        #######


                            if y - 1 >= 0:        
                                if new_ima[x][y - 1] == -1:
                                    new_ima[x][y - 1] = region.label + num_of_label
                                    gap_num[x,y - 1] = nowGapsNum
                                    counter_num[x, y - 1] = counter
                                    if (x, y - 1) in nexpGap_lab:
                                        nexpGap_lab.remove((x, y - 1))
                                    bt_lab.append((x, y - 1))
                                    flag_hasNew = True
                                    #if m in bt_lab:
                                        #bt_lab.remove(m)
                                #print(len(tem_lab))
                                elif new_ima[x][y - 1] > num_of_label and new_ima[x][y - 1] != region.label + num_of_label:
                                    if adjacent_m[region.label][new_ima[x][y - 1] - num_of_label] == -1000 or adjacent_m[region.label][new_ima[x][y - 1] - num_of_label] > counter:
                                        adjacent_m[region.label][new_ima[x][y - 1] - num_of_label] = counter
                                        adjacent_m[new_ima[x][y - 1] - num_of_label][region.label] = counter
                                        #######
                                        adjacent_num[region.label][new_ima[x][y-1] - num_of_label] = adjacent_num[region.label][new_ima[x][y-1] - num_of_label]+1
                                        adjacent_num[new_ima[x][y-1] - num_of_label][region.label] = adjacent_num[new_ima[x][y-1] - num_of_label][region.label]+1           
                                        #######
                        traces = copy.deepcopy(bt_lab)


                    counter += 5    
                    traces = copy.deepcopy(nexpGap_lab)
                    bt_lab = copy.deepcopy(nexpGap_lab)

            if TEST_SWITCH_R and region.label == TEST_NEU:
                plt.figure(figsize = [35, 35])
                plt.imshow(new_ima)
                plt.colorbar()

                plt.figure(figsize = [35, 35])
                plt.imshow(counter_num)
                plt.colorbar()
                plt.imshow(new_ima, alpha = 0.25)
                plt.savefig('counter_3D.png')
                break

        stop = time()
        print("\nProcessing time "+str(stop-start) + "sec")
        #new_image_list.append(new_ima)
        adjacent_m_list.append(adjacent_m)
        #num_of_label_list.append(num_of_label)
        #plt.figure(figsize=[50, 50])
        #plt.imshow(new_ima)
        #plt.colorbar()
        break





    adjacent_m=adjacent_m_list[0][1:,1:]
    minv=3000
    for m in range(0, adjacent_m.shape[0]):
        for n in range(0, adjacent_m.shape[1]):
            if adjacent_m[m][n]>0:
                if adjacent_m[m][n]<minv:
                    minv=adjacent_m[m][n]
    i=0
    for m in range(0, adjacent_m.shape[0]):
        for n in range(0, adjacent_m.shape[1]):
            if adjacent_m[m][n]==-1:
                adjacent_m[m][n]=minv
                i=i+1

    #7: plot the adjacent matrix of 0th image
    plt.figure(figsize=[25, 25])
    plt.imshow(adjacent_m_list[0][1:,1:])
    plt.colorbar()
    plt.show()
    #plt.savefig('matrix_3D.png')
    np.save(path + 'results/' + saved_name + '_adj(length)_'+scale_str+'.npy', adjacent_m)
    #XC: For test only 
#     plt.figure(figsize = [25, 25])
#     plt.imshow(counter_num) 
#     plt.colorbar()



In [ ]:
int(round(a1.shape[0]*scale[1]//6))

In [ ]:
a2.shape[1]